In [ ]:
import numpy as np
import random
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def generate_password(strength):
    if strength == "weak":
        return ''.join(random.choices(string.ascii_lowercase, k=random.randint(4, 6)))
    elif strength == "medium":
        return ''.join(random.choices(string.ascii_letters + string.digits, k=random.randint(6, 10)))
    elif strength == "strong":
        return ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation, k=random.randint(10, 16)))


In [ ]:
def generate_dataset(n_per_class=1000):
    passwords = []
    labels = []
    for _ in range(n_per_class):
        for strength in ["weak", "medium", "strong"]:
            passwords.append(generate_password(strength))
            labels.append(strength)
    return passwords, labels

In [ ]:
def preprocess_data(passwords, labels, maxlen=16):
    tokenizer = Tokenizer(char_level=True, lower=False)
    tokenizer.fit_on_texts(passwords)
    X_seq = tokenizer.texts_to_sequences(passwords)
    X_pad = pad_sequences(X_seq, maxlen=maxlen, padding='post')

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    return X_pad, y, tokenizer, label_encoder

In [ ]:
def build_model(vocab_size, input_length):
    model = Sequential([
        Embedding(input_dim=vocab_size + 1, output_dim=32, input_length=input_length),
        LSTM(64, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes: weak, medium, strong
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train():
    passwords, labels = generate_dataset()
    X, y, tokenizer, label_encoder = preprocess_data(passwords, labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = build_model(vocab_size=len(tokenizer.word_index), input_length=X.shape[1])
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, verbose=1)

    y_pred = np.argmax(model.predict(X_test), axis=1)
    print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


In [ ]:
if __name__ == "__main__":
    train()